In [ ]:
from autosted import AcquisitionPipeline
from autosted.taskgeneration import AcquisitionTaskGenerator
from autosted.callback_buildingblocks.static_settings import JSONSettingsLoader
from autosted.callback_buildingblocks.parameter_filtering import LocationRemover
from autosted.callback_buildingblocks.regular_position_generators import SpiralOffsetGenerator
from autosted.imspector import get_current_stage_coords
from autosted.stoppingcriteria import MaximumAcquisitionsStoppingCriterion

import logging
logging.basicConfig(level=logging.INFO)

# Overview-Detail imaging with **autoSTED**

**Note:** Imspector should be open in the background.

Let's start with what we had at the end of ```basics.ipynb``` notebook, running overview acquisitions in a spiral.

Here, we added a few small changes:
* instead of using the parameters of the current measurement, we use parameters savved to a json file via ```save_parameters_to_json.ipynb```
* we wrap the settings loader in a ```LocationRemover```: this just removes any location-related parameters (i.e. stage/scan offsets)
    * this is not strictly necessary here, as those parameters will be overwritten by the ```SpiralOffsetGenerator```, but to keep everything clean, it still makes sense.

**Note:** When building your own pipeline, you could also build upon the other ```overview_*``` notebooks in the examples folder for:
* imaging in a regular grid (optionally with on-the-fly stitching)
* imaging at manually picked locations
* image-based autofocus in the overviews
* selective overview imaging with a pre-scan

In [ ]:
# pipeline with a single overview level: "image" 
pipeline = AcquisitionPipeline(data_save_path='acquisition_data/test', hierarchy_levels=['image'])

# path to parameters saved as JSON
overview_config = 'config_json/test2color_overview.json'

# overview generator combines settings from file with next stage positions in spiral
next_overview_generator = AcquisitionTaskGenerator('image',
                         LocationRemover(JSONSettingsLoader(overview_config)),
                         SpiralOffsetGenerator(move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()))

# add the callback and a stopping condition
pipeline.add_callback(next_overview_generator, 'image')
pipeline.add_stopping_condition(MaximumAcquisitionsStoppingCriterion(5))

pipeline.run(initial_callback=next_overview_generator)

## Accessing the data of a run

An ```AcquisitionPipeline``` stores acquired data (and the parameters used) in its ```.data``` attribute, which acts like a ```dict```.

The keys are tuple of ints, a running count for each hierarchy level. E.g. the first overview image has index ```(0, )```. If we also do detail acquisitions, the second detail in the third overview would have index ```(2, 1)```, ...

We can get a single ```MeasurementData``` object, which contains lists of data, measurement parameters and hardware parameters (for each *configuration* of the measurement). The data themselves are a list of NumPy arrays for the different channels of the acquisition.

In [ ]:
# get data of a given index
measurement_data = pipeline.data[(0,)]

# get data of configuration 0, channel 0, squeeze singleton dimensions (Imspector stacks are always 4D)
img = measurement_data.data[0][0].squeeze()

We can plot the image with matplotlib:

In [ ]:
from matplotlib import pyplot as plt

# NOTE: this assumes you did a 2D overview
# for 3D data, you would have to e.g., max project it along the z-axis
# img = img.max(axis=0)

plt.imshow(img, cmap='magma')

## Segmenting cells

To build an automation pipeline that selectively images cells in the overview with higher resolution, we first need a segmentation function that takes an image (NumPy array) and returns an integer-valued label map of the same shape.

We can use standard Python image processing functionality from libraries like ```scikit-image``` or ```scipy```:

In [16]:
from skimage.filters import threshold_otsu
from scipy.ndimage import gaussian_filter, label
from skimage.segmentation import clear_border
from skimage.morphology import dilation, disk


def segment(img):
    # blur and get Otsu threshold
    g = gaussian_filter(img.astype(float), 5)
    t = max(3, threshold_otsu(g))
    # label connected components, remove objects at border, dilate
    labels, _ = label(g > t)
    labels = clear_border(labels)
    labels = dilation(labels, disk(3))
    return labels

We can test our function on the image we got from the pipeline earlier & plot the results:

In [ ]:
label_map = segment(img)

plt.imshow(label_map, cmap='turbo', interpolation='nearest')

### Advanced segmentation

The segmentation function does not have any specific dependencies to autoSTED, so you can use pretty much anything in the Python image processing ecosystem, e.g. deep learning-based segmentation via Cellpose (https://github.com/MouseLand/cellpose)

In [ ]:
from cellpose.models import Cellpose

# instantiate Cellpose model
model = Cellpose(gpu=True, model_type='nuclei')

def segment_cellpose(img, diameter=30, model=model):
    # run model, return predicted instance segmentation mask     
    masks, flows, styles, diams = model.eval([img], diameter=diameter, flow_threshold=None, channels=[0,0])
    return masks[0]

label_map = segment_cellpose(img)
plt.imshow(label_map, cmap='turbo', interpolation='nearest')

## Using the segmentation function for an overview-detail pipeline

To move from the single-level automation pipeline above to an overview-detail pipeline, we just have to add a second callback to it that will be called after each overview image and enqueue details.

Again, we can construct it from simple building blocks using an ```AcquisitionTaskGenerator```:

* 1. get base settings from a JSON File
* 2. take the location of the overview image (esp. stage position)
* 3. use our segmentation function wrapped in a ```SegmentationWrapper``` - this will apply the function to the newest image that was acquired, translate the pixel objects into scan offset & ROI size parameters.

In [ ]:
from autosted.callback_buildingblocks.parameter_filtering import LocationKeeper
from autosted.callback_buildingblocks.data_selection import NewestSettingsSelector
from autosted.detection.roi_detection import SegmentationWrapper

# pipeline and overview generator as above, but we now have two levels: 'overview', 'detail'
pipeline = AcquisitionPipeline(data_save_path='acquisition_data/test', hierarchy_levels=['overview', 'detail'])

overview_config = 'config_json/test2color_overview.json'
next_overview_generator = AcquisitionTaskGenerator('overview',
                         JSONSettingsLoader(overview_config),
                         SpiralOffsetGenerator(move_size=[50e-6, 50e-6], start_position=get_current_stage_coords()))

# acquisition task generator for details as described above
detail_config = "config_json/test2color_detail.json"
detail_generator = AcquisitionTaskGenerator(
    "detail",
    # 1. base settings from file
    LocationRemover(JSONSettingsLoader(detail_config)),
    # 2. locations (stage) from previous (overview) image
    LocationKeeper(NewestSettingsSelector()),
    # 3. segmentation wrapper around segmentation function
    SegmentationWrapper(segment),
)

pipeline.add_callback(next_overview_generator, "overview")
pipeline.add_callback(detail_generator, "overview")

pipeline.add_stopping_condition(
    # instead of a maximum of total images, we can also specify a maximum per level
    MaximumAcquisitionsStoppingCriterion(
        max_acquisitions_per_level={"overview": 5, "detail": 20}
    )
)

pipeline.run(initial_callback=next_overview_generator)

### SegmentationWrapper Internals

The SegmentationWrapper object takes care of getting image data as NumPy array(s), passing it to our segmentation function and translating the pixel results back to physical microscope parameters (scan offsets, FOV lengths) and wrapping them as parameter ```dict```s.

We can adjust the behavious via the constructor, e.g.:
* how to get the measurement data in which to perform detection (by default, we select the newest image of the level at which the callback is attached)
* which configuration(s) and channel(s) to use
* whether to plot results
* wheter to return scan or stage offsets
* whether to return a ready-to-use parameter dictionary (default) or just a list of bounding boxes, which may be isnteresting if we want to e.g., manually add an offset. In the latter case, we can nest the SegmentationWrapper in a ```ScanOffsetsSettingsGenerator``` or ```StageOffsetsSettingsGenerator``` to wrap the results into a parameter dict.

In [ ]:
from autosted.callback_buildingblocks.coordinate_value_wrappers import ScanOffsetsSettingsGenerator
from autosted.callback_buildingblocks.data_selection import NewestDataSelector

detail_generator = AcquisitionTaskGenerator(
    "detail",
    LocationRemover(JSONSettingsLoader("config_json/test2color_detail.json")),
    LocationKeeper(NewestSettingsSelector()),
    ScanOffsetsSettingsGenerator(
        SegmentationWrapper(
            segment,
            data_source_callback=NewestDataSelector(pipeline=pipeline, level="overview"),
            configurations=0,
            channels=0,
            offset_parameters='scan',
            plot_detections=True,
            return_parameter_dict=False            
        )
    ),
)

## Adding on-the-fly stitching of overviews

The modular nature of autoSTED allows for easy exchange of building blocks, e.g. the data source callback of our segmentation wrapper could be exchanged for a ```StitchedNewestDataSelector``` to virtually stitch overview images and thus prevent skipping of cells on the border of overview tiles.

One issue that might arise here is that since the cell detector "sees the same overview multiple times" cells might be selected for detailled imagin multiple times. To prevent this, we have an ```AlreadyImagedFOVFilter``` that can be attached to the acquisition task generator to skip already imaged FOVs.

In [ ]:
from autosted.taskgeneration.task_filtering import AlreadyImagedFOVFilter
from autosted.callback_buildingblocks.stitched_data_selection import StitchedNewestDataSelector

pipeline = AcquisitionPipeline(
    data_save_path="acquisition_data/test", hierarchy_levels=["overview", "detail"]
)

overview_config = 'config_json/test2color_overview.json'
next_overview_generator = AcquisitionTaskGenerator(
    "overview",
    JSONSettingsLoader(overview_config),
    SpiralOffsetGenerator(
        # NOTE: smaller move size to have overlap
        move_size=[40e-6, 40e-6], start_position=get_current_stage_coords()
    ),
)

detail_config = "config_json/test2color_detail.json"
detail_generator = AcquisitionTaskGenerator(
    "detail",
    LocationRemover(JSONSettingsLoader(detail_config)),
    LocationKeeper(NewestSettingsSelector(pipeline, "overview")),
    SegmentationWrapper(
        segment,
        # instead of default NewestDataSelector, we use StitchedNewestDataSelector
        # which returns a virtually stitched image of the most recent overview and its neighbors
        data_source_callback=StitchedNewestDataSelector(
            pipeline, "overview", register_tiles=False, offset_parameters="scan"
        ),
        offset_parameters="scan",
    ),
)
# we add a task filter to ignore FOVs already imaged at "detail" level
detail_generator.add_task_filters(AlreadyImagedFOVFilter(pipeline, "detail", 0.5, True))


pipeline.add_callback(next_overview_generator, "overview")
pipeline.add_callback(detail_generator, "overview")

pipeline.add_stopping_condition(
    MaximumAcquisitionsStoppingCriterion(
        max_acquisitions_per_level={"overview": 5, "detail": 20}
    )
)

pipeline.run(initial_callback=next_overview_generator)